In [ ]:
# IMPORT LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization, GlobalAveragePooling2D, LeakyReLU, SpatialDropout2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import AdamW
from sklearn.utils.class_weight import compute_class_weight

from src.config import image_classes

print("✅ Libraries installed and imported.")

In [ ]:
data = np.load('/emotion_detection_data.npz')

In [ ]:
x_train = data['x_train']
y_train = data['y_train']
x_val = data['x_val']
y_val = data['y_val']
x_test = data['x_test']
y_test = data['y_test']

In [ ]:
def evaluation_function(history, model):
    """
    Evaluates a trained Keras model by plotting training history, calculating
    accuracy on train, validation, and test sets, and visualizing the confusion matrix.

    This function expects `x_test`, `y_test`, `x_train`, `y_train`, `x_val`,
    `y_val`, and `image_classes` to be globally accessible or passed implicitly
    from the surrounding scope.

    Args:
        history (keras.callbacks.History): A History object returned by the `fit`
            method of a Keras model. It contains the loss and metrics values
            during training.
        model (keras.Model or str): The Keras model object itself, or the path
            to the saved weights file of the best performing model. The function
            will load weights into `history.model` if `model` is a path.

    Plots:
        - **Model Loss:** A plot showing training loss and validation loss over epochs.
        - **Model Accuracy:** A plot showing training accuracy and validation accuracy over epochs.
        - **Confusion Matrix:** A heatmap visualizing the confusion matrix for
          predictions on the test set.

    Prints:
        - The test accuracy of the submitted model.
        - The train accuracy of the submitted model.
        - The validation accuracy of the submitted model.
    """
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    history.model.load_weights(model)
    test_acc = np.mean(np.argmax(history.model.predict(x_test),1)==np.argmax(y_test,1))
    print('The submitted model has test accuracy equal to {}'.format(test_acc))

    train_acc = np.mean(np.argmax(history.model.predict(x_train),1)==np.argmax(y_train,1))
    print('The submitted model has train accuracy equal to {}'.format(train_acc))

    val_acc = np.mean(np.argmax(history.model.predict(x_val),1)==np.argmax(y_val,1))
    print('The submitted model has validation accuracy equal to {}'.format(val_acc))

    y_pred_probs = model.predict(x_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Confusion matrix visualization
    conf_matrix = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=image_classes, yticklabels=image_classes)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

In [ ]:
classes = 7
batch = 16
optimizer = AdamW(learning_rate=0.0001, weight_decay=0.0001)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train.argmax(axis=1)), y=y_train.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

In [ ]:
# 1st custom cnn Model

model_1 = Sequential([

    Conv2D(32, (3,3), padding='same', strides=(1,1), activation='relu', input_shape=(48,48,1)),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.2),

    Conv2D(64, (3,3), padding='same', strides=(1,1), activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.2),

    Conv2D(128, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.3),

    Conv2D(256, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.3),

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(7, activation='softmax')
])

In [ ]:
epochs = 50

save_model = ModelCheckpoint('custom_cnn_1.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy',  patience=10,  restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.00001, verbose=1)
model_1.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model_1.fit(x_train, y_train, batch_size=batch, epochs=epochs, validation_data=(x_val, y_val), shuffle = True, verbose=1,
                    callbacks=[save_model, early_stop, lr_scheduler], class_weight = class_weights)

evaluation_function(history, 'custom_cnn_1.keras')

In [ ]:
model_2 = Sequential([

    Conv2D(32, (3,3), padding='same', strides=(1,1), activation='linear', input_shape=(48,48,1)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(32, (3,3), padding='same', strides=(1,1), activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.20),

    Conv2D(64, (3,3), padding='same', strides=(1,1), activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', strides=(1,1), activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.25),

    Conv2D(128, (3,3), padding='same', activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same', activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D((2,2)),
    SpatialDropout2D(0.3),

    Flatten(),
    Dense(256, activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

print(model_2.summary())

In [ ]:
epochs = 60

save_model = ModelCheckpoint('custom_cnn_2.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=2)
model_2.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history_2 = model_2.fit(x_train, y_train, batch_size=batch, epochs=epochs, validation_data=(x_val, y_val), shuffle = True, verbose=1,
                    callbacks=[save_model, early_stop, lr_scheduler], class_weight = class_weights)

evaluation_function(history_2, 'custom_cnn_2.keras')

In [ ]:
model_3 = Sequential([

    Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='he_normal', input_shape=(48,48,1)),
    BatchNormalization(),
    Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),
    Dropout(0.2),

    Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),
    Dropout(0.2),

    Conv2D(128, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),
    Dropout(0.3),

    Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),
    Dropout(0.3),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(512, activation='linear'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

print(model_3.summary())

In [ ]:
epochs = 30

save_model = ModelCheckpoint('custom_cnn_3.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=2)
model_3.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history_3 = model_3.fit(x_train, y_train, batch_size=batch, epochs=epochs, validation_data=(x_val, y_val), shuffle = True, verbose=1,
                    callbacks=[save_model, early_stop, lr_scheduler], class_weight = class_weights)

evaluation_function(history_3, 'custom_cnn_3.keras')

In [ ]:
model_4 = Sequential([

    Conv2D(64, (3,3), padding='same',kernel_initializer='he_normal', activation='relu', input_shape=(48,48,1)),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),

    Conv2D(128, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),

    Conv2D(256, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),
    SpatialDropout2D(0.2),

    Conv2D(512, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(512, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    Conv2D(512, (3,3), padding='same',kernel_initializer='he_normal', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2,2), strides=(2,2)),
    SpatialDropout2D(0.2),

    Flatten(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dense(7, activation='softmax')
])

print(model_4.summary())

In [ ]:
epochs = 25

save_model = ModelCheckpoint('custom_cnn_4.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=2)
model_4.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model_4.fit(x_train, y_train, batch_size=batch, epochs=epochs, validation_data=(x_val, y_val), shuffle = True, verbose=1,
                    callbacks=[save_model, early_stop, lr_scheduler], class_weight = class_weights)

evaluation_function(history_3, 'custom_cnn_4.keras')